In [5]:
import numpy as np
import pandas as pd
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

In [6]:
# reviews = json.load(open("../data/IMDB_reviews.json"))
movie_details = []
with open("../data/IMDB_movie_details.json") as jsonfile:
    for jsonf in jsonfile:
        movie_details.append(json.loads(jsonf))

In [7]:
df_movie_details = pd.json_normalize(movie_details)

In [8]:
# reviews = json.load(open("../data/IMDB_reviews.json"))
reviews = []
with open("../data/IMDB_reviews.json") as jsonfile:
    for jsonf in jsonfile:
        reviews.append(json.loads(jsonf))

In [9]:
df_reviews = pd.json_normalize(reviews)

In [10]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573913 entries, 0 to 573912
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   review_date     573913 non-null  object
 1   movie_id        573913 non-null  object
 2   user_id         573913 non-null  object
 3   is_spoiler      573913 non-null  bool  
 4   review_text     573913 non-null  object
 5   rating          573913 non-null  object
 6   review_summary  573913 non-null  object
dtypes: bool(1), object(6)
memory usage: 26.8+ MB


In [11]:
df_reviews.head()

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"


In [12]:
df_reviews.isnull().sum()

review_date       0
movie_id          0
user_id           0
is_spoiler        0
review_text       0
rating            0
review_summary    0
dtype: int64

In [13]:
df_reviews["is_spoiler"].value_counts()

is_spoiler
False    422989
True     150924
Name: count, dtype: int64

In [14]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(df_reviews['review_text'])

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_reviews['review_text'], df_reviews['is_spoiler'].astype('int'), test_size=0.2)

In [16]:
train_token = tokenizer.texts_to_sequences(X_train)
test_token = tokenizer.texts_to_sequences(X_test)

In [17]:
padded_train = pad_sequences(train_token, maxlen=20)
padded_test = pad_sequences(test_token, maxlen=20)

In [18]:
model = tf.keras.models.Sequential([tf.keras.layers.Embedding(2000, 128, input_length=20),
                                    tf.keras.layers.LSTM(128),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid'),
                                   ])

2023-10-30 12:59:33.763904: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 12:59:33.805409: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 12:59:33.805471: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 12:59:33.809365: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-30 12:59:33.809439: I tensorflow/compile

In [19]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer='adam',
             metrics=['accuracy'])

In [1]:
model.fit(padded_train, y_train, validation_data=(padded_test, y_test), epochs=1)

NameError: name 'model' is not defined

In [ ]:
loss, accuracy = model.evaluate(padded_train, y_train, verbose = False)
accuracy